In [12]:
!pip install transformers gradio --quiet
import gradio as gr
from transformers import pipeline
import random
import re


In [13]:
caption_generator = pipeline("text-generation", model="gpt2")
sentiment_pipeline = pipeline("sentiment-analysis")

emoji_dict = {
    "positive": [
        "😃", "😊", "😁", "😎", "👍", "❤️", "🎉", "✨", "🌟", "🥳",
        "🤩", "😍", "👌", "💪", "🤗", "😄", "😺", "🌈", "🍀", "🎈",
        "🥰", "🍓", "🍰", "🕺", "💃", "🎶", "🎵", "☀️", "🌸", "🐱"
    ],
    "negative": [
        "😢", "☹️", "😠", "💔", "😞", "😩", "😭", "😡", "👎", "😖",
        "😣", "😤", "🥀", "😿", "💀", "🥺", "😰", "😨", "👹", "😓",
        "😫", "🤯", "👎🏿", "🖤", "😟", "🧟", "👺", "☠️", "⚰️", "😬"
    ],
    "neutral": [
        "😐", "😶", "🤔", "🙂", "😌", "😑", "😬", "😴", "😕", "🙃",
        "😷", "🤨", "😳", "😶‍🌫️", "🤫", "🧐", "😒", "🧏", "🙄", "🛌",
        "🎭", "📊", "🎬", "🗿", "📚", "📰", "🤥", "😈", "🤡", "👻"
    ]
}

def get_emojis(text):
    if not text.strip():
        return "🙂 🙂 🙂"

    result = sentiment_pipeline(text[:512])[0]
    label = result['label'].lower()
    emojis = emoji_dict.get(label, emoji_dict["neutral"])

    sample_size = min(3, len(emojis))
    selected_emojis = random.sample(emojis, sample_size)
    return ' '.join(selected_emojis)

Device set to use cpu
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


In [16]:
def get_hashtags(prompt, platform):
    words = prompt.lower().split()
    tags = ["#" + word.replace(" ", "").replace(",", "") for word in words if len(word) > 3]

    platform_tags = {
        "Instagram": [
            "#instadaily", "#igers", "#picoftheday", "#photooftheday",
            "#love", "#beautiful", "#fashion", "#happy", "#cute",
            "#followme", "#tbt", "#smile", "#instalike", "#like4like",
            "#nature", "#travel", "#style", "#selfie", "#summer", "#art"
        ],
        "LinkedIn": [
            "#career", "#leadership", "#networking", "#professional",
            "#business", "#success", "#motivation", "#growth",
            "#entrepreneur", "#innovation", "#teamwork", "#jobsearch",
            "#work", "#digitaltransformation", "#careerdevelopment",
            "#remotework", "#strategy", "#productivity", "#inspiration"
        ],
        "Twitter": [
            "#tweet", "#trending", "#news", "#breakingnews", "#viral",
            "#followfriday", "#MondayMotivation", "#TechNews", "#SocialMedia",
            "#update", "#breaking", "#ICYMI", "#funny", "#retweet",
            "#hashtag", "#trends", "#politics", "#sports", "#weather",
            "#entertainment"
        ]
    }

    chosen_platform_tags = platform_tags.get(platform, [])
    sample_size = min(3, len(chosen_platform_tags))
    platform_sample = random.sample(chosen_platform_tags, sample_size)

    return " ".join(tags[:4] + platform_sample)


In [17]:
def create_better_prompt(keyword, platform):
    """Create platform-specific prompts for better caption generation"""
    platform_prompts = {
        "Instagram": f"Create an inspiring Instagram caption about {keyword}:",
        "LinkedIn": f"Write a professional LinkedIn post about {keyword}:",
        "Twitter": f"Compose a catchy tweet about {keyword}:"
    }
    return platform_prompts.get(platform, f"Write about {keyword}:")

def clean_generated_text(text, original_prompt):
    """Clean and format the generated text"""
    if text.startswith(original_prompt):
        text = text[len(original_prompt):].strip()
    text = re.sub(r'^[\s\-\.\,\:]+', '', text)
    sentences = re.split(r'[.!?\n]', text)
    if sentences:
        clean_text = sentences[0].strip()
        if len(clean_text) > 10:
            return clean_text
    return text[:100].strip()

In [21]:
def generate_post(prompt, platform):
    if not prompt.strip():
        return "Please enter a keyword or theme", "🙂 🙂 🙂", "#social #media"

    # Create better prompt for caption generation
    formatted_prompt = create_better_prompt(prompt, platform)

    try:
        # Generate text with better parameters
        gen_output = caption_generator(
            formatted_prompt,
            max_length=80,
            min_length=20,
            num_return_sequences=1,
            temperature=0.8,
            do_sample=True,
            pad_token_id=caption_generator.tokenizer.eos_token_id
        )[0]['generated_text']
        caption = clean_generated_text(gen_output, formatted_prompt)
        if len(caption) < 20:
            caption = f"Discover the amazing world of {prompt}! Check this out!"

    except Exception as e:
        caption = f"Exciting news about {prompt}! Stay tuned for more updates."

    emojis = get_emojis(caption)
    hashtags = get_hashtags(prompt, platform)

    return caption, emojis, hashtags


In [22]:
interface = gr.Interface(
    fn=generate_post,
    inputs=[
        gr.Textbox(label="Enter keyword or theme", placeholder="e.g., travel, technology, food"),
        gr.Radio(["Instagram", "LinkedIn", "Twitter"], label="Choose Platform", value="Instagram")
    ],
    outputs=[
        gr.Textbox(label="Generated Caption", lines=3),
        gr.Textbox(label="Emojis"),
        gr.Textbox(label="Hashtags", lines=2)
    ],
    title="Social Media Caption & Emoji Generator",
    description="Generate engaging captions, relevant hashtags, and emojis for your social media posts!"
)

interface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6149b3f9dd23303aa9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install transformers gradio --quiet

import gradio as gr
from transformers import pipeline
import random
import re

# Load pipelines once
caption_generator = pipeline("text-generation", model="gpt2")
sentiment_pipeline = pipeline("sentiment-analysis")

emoji_dict = {
    "positive": [
        "😃", "😊", "😁", "😎", "👍", "❤️", "🎉", "✨", "🌟", "🥳",
        "🤩", "😍", "👌", "💪", "🤗", "😄", "😺", "🌈", "🍀", "🎈",
        "🥰", "🍓", "🍰", "🕺", "💃", "🎶", "🎵", "☀️", "🌸", "🐱"
    ],
    "negative": [
        "😢", "☹️", "😠", "💔", "😞", "😩", "😭", "😡", "👎", "😖",
        "😣", "😤", "🥀", "😿", "💀", "🥺", "😰", "😨", "👹", "😓",
        "😫", "🤯", "👎🏿", "🖤", "😟", "🧟", "👺", "☠️", "⚰️", "😬"
    ],
    "neutral": [
        "😐", "😶", "🤔", "🙂", "😌", "😑", "😬", "😴", "😕", "🙃",
        "😷", "🤨", "😳", "😶‍🌫️", "🤫", "🧐", "😒", "🧏", "🙄", "🛌",
        "🎭", "📊", "🎬", "🗿", "📚", "📰", "🤥", "😈", "🤡", "👻"
    ]
}

def get_emojis(text):
    if not text.strip():
        return "🙂 🙂 🙂"

    result = sentiment_pipeline(text[:512])[0]
    label = result['label'].lower()
    emojis = emoji_dict.get(label, emoji_dict["neutral"])

    sample_size = min(3, len(emojis))
    selected_emojis = random.sample(emojis, sample_size)
    return ' '.join(selected_emojis)

def get_hashtags(prompt, platform):
    words = prompt.lower().split()
    tags = ["#" + word.replace(" ", "").replace(",", "") for word in words if len(word) > 3]

    platform_tags = {
        "Instagram": [
            "#instadaily", "#igers", "#picoftheday", "#photooftheday",
            "#love", "#beautiful", "#fashion", "#happy", "#cute",
            "#followme", "#tbt", "#smile", "#instalike", "#like4like",
            "#nature", "#travel", "#style", "#selfie", "#summer", "#art"
        ],
        "LinkedIn": [
            "#career", "#leadership", "#networking", "#professional",
            "#business", "#success", "#motivation", "#growth",
            "#entrepreneur", "#innovation", "#teamwork", "#jobsearch",
            "#work", "#digitaltransformation", "#careerdevelopment",
            "#remotework", "#strategy", "#productivity", "#inspiration"
        ],
        "Twitter": [
            "#tweet", "#trending", "#news", "#breakingnews", "#viral",
            "#followfriday", "#MondayMotivation", "#TechNews", "#SocialMedia",
            "#update", "#breaking", "#ICYMI", "#funny", "#retweet",
            "#hashtag", "#trends", "#politics", "#sports", "#weather",
            "#entertainment"
        ]
    }

    chosen_platform_tags = platform_tags.get(platform, [])
    sample_size = min(3, len(chosen_platform_tags))
    platform_sample = random.sample(chosen_platform_tags, sample_size)

    return " ".join(tags[:4] + platform_sample)

def create_better_prompt(keyword, platform):
    """Create platform-specific prompts for better caption generation"""
    platform_prompts = {
        "Instagram": f"Create an inspiring Instagram caption about {keyword}:",
        "LinkedIn": f"Write a professional LinkedIn post about {keyword}:",
        "Twitter": f"Compose a catchy tweet about {keyword}:"
    }
    return platform_prompts.get(platform, f"Write about {keyword}:")

def clean_generated_text(text, original_prompt):
    """Clean and format the generated text"""
    # Remove the original prompt
    if text.startswith(original_prompt):
        text = text[len(original_prompt):].strip()

    # Remove leading punctuation and whitespace
    text = re.sub(r'^[\s\-\.\,\:]+', '', text)

    # Take only the first sentence or up to first period/newline
    sentences = re.split(r'[.!?\n]', text)
    if sentences:
        clean_text = sentences[0].strip()
        if len(clean_text) > 10:  # Ensure meaningful content
            return clean_text

    # Fallback: return first 100 characters
    return text[:100].strip()

def generate_post(prompt, platform):
    if not prompt.strip():
        return "Please enter a keyword or theme", "🙂 🙂 🙂", "#social #media"

    # Create better prompt for caption generation
    formatted_prompt = create_better_prompt(prompt, platform)

    try:
        # Generate text with better parameters
        gen_output = caption_generator(
            formatted_prompt,
            max_length=80,
            min_length=20,
            num_return_sequences=1,
            temperature=0.8,
            do_sample=True,
            pad_token_id=caption_generator.tokenizer.eos_token_id
        )[0]['generated_text']

        # Clean the generated caption
        caption = clean_generated_text(gen_output, formatted_prompt)

        # Fallback if caption is too short
        if len(caption) < 10:
            caption = f"Discover the amazing world of {prompt}! Check this out!"

    except Exception as e:
        caption = f"Exciting news about {prompt}! Stay tuned for more updates."

    emojis = get_emojis(caption)
    hashtags = get_hashtags(prompt, platform)

    return caption, emojis, hashtags

interface = gr.Interface(
    fn=generate_post,
    inputs=[
        gr.Textbox(label="Enter keyword or theme", placeholder="e.g., travel, technology, food"),
        gr.Radio(["Instagram", "LinkedIn", "Twitter"], label="Choose Platform", value="Instagram")
    ],
    outputs=[
        gr.Textbox(label="Generated Caption", lines=3),
        gr.Textbox(label="Emojis"),
        gr.Textbox(label="Hashtags", lines=2)
    ],
    title="Social Media Post & Caption Generator",
    description="Generate engaging captions, relevant hashtags, and emojis for your social media posts!"
)

interface.launch()
